In [2]:
import functools

### Decorator without arguments

In [3]:
def decorator_wo_args(original_function):

    def wrapped_function(*args, **kwargs):
        print('args:', args, 'kwargs:', kwargs)
        out = original_function(*args, **kwargs)
        return out
        
    return wrapped_function

In [4]:
@decorator_wo_args
def func_dec_wo_args(func_arg='!'):
    print('func_dec_wo_args', func_arg)

In [5]:
func_dec_wo_args()

args: () kwargs: {}
func_dec_wo_args !


### Decorator with arguments

In [6]:
def decorator_w_args(argument):

    def _decorate(function):
        
        # functools fixes function metadata (e.g. name)
        @functools.wraps(function)
        def wrapped_function(*args, **kwargs):
            print('args:', args, 'kwargs:', kwargs)
            print('decorator argument:', argument)
            out = function(*args, **kwargs)
            return out
        
        return wrapped_function

    return _decorate

In [7]:
@decorator_w_args(42)
def func_dec_w_args(func_arg='!'):
    print('func_dec_w_args', func_arg)

In [8]:
func_dec_w_args(func_arg='?')

args: () kwargs: {'func_arg': '?'}
decorator argument: 42
func_dec_w_args ?


### Decorator with or without arguments

In [9]:
def decorator_w_or_wo_args(original_function=None, optional_argument=None):

    def _decorate(function):

        # functools fixes function metadata (e.g. name)
        @functools.wraps(function)
        def wrapped_function(*args, **kwargs):
            print('args:', args, 'kwargs:', kwargs)
            print('decorator argument:', optional_argument)
            out = function(*args, **kwargs)
            return out
        
        return wrapped_function

    if original_function:
        return _decorate(original_function)
    else:
        return _decorate

In [10]:
@decorator_w_or_wo_args
def func_dec_w_or_wo_args1(func_arg='!'):
    print('func_dec_w_or_wo_args1', func_arg)

@decorator_w_or_wo_args(optional_argument=99)
# in this case it the decorator argument must have a keyword
def func_dec_w_or_wo_args2(func_arg='!'):
    print('func_dec_w_or_wo_args2', func_arg)

In [11]:
func_dec_w_or_wo_args1('!?')

args: ('!?',) kwargs: {}
decorator argument: None
func_dec_w_or_wo_args1 !?


In [12]:
func_dec_w_or_wo_args2()

args: () kwargs: {}
decorator argument: 99
func_dec_w_or_wo_args2 !


### Example from NetworkUnit
If there exists a pre-calculated prediction in cache return it without recalculating,
else calculate it and save it to the cache.
Optionally, a hash for the cache location can be provided.

In [ ]:
def use_prediction_cache(generate_prediction_func=None, hash_key=None):
    """
    Decorator for the `generate_prediction()` function of the tests, handles
    cached prediction loading, parameter update and prediction saving.
    """

    def _decorate(function):

        @functools.wraps(function)
        def wrapper(self, model):

            # Check if predictions were already calculated
            prediction = self.get_prediction(model, key=hash_key)

            # If any parameter was specified by the user in the generate_prediction
            # function the predictions are recalculated
            if prediction is None:

                # Generate and save prediction
                prediction = function(self, model)
                self.set_prediction(model, prediction, key=hash_key)

            return prediction

        return wrapper

    if generate_prediction_func:
        return _decorate(generate_prediction_func)
    else:
        return _decorate